In [21]:
# from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import pickle
import torchvision
from torchvision import transforms
import torchvision.datasets as dset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import numpy as np
import sys
from collections import deque
import os

In [2]:
# vocabulary = []
# for sentence in tokenized_corpus:
#     for token in sentence:
#         if token not in vocabulary:
#             vocabulary.append(token)

vocabulary = ["anastasia", "valentin", "marko", "sophie", "rene", "claudia", 
              "lorenz", "hannah", "paula", "nico", "dominik", "nora"]

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)
print (vocabulary)
print ("vocabulary_size: " + str(vocabulary_size))
print (word2idx)

['anastasia', 'valentin', 'marko', 'sophie', 'rene', 'claudia', 'lorenz', 'hannah', 'paula', 'nico', 'dominik', 'nora']
vocabulary_size: 12
{'anastasia': 0, 'valentin': 1, 'marko': 2, 'sophie': 3, 'rene': 4, 'claudia': 5, 'lorenz': 6, 'hannah': 7, 'paula': 8, 'nico': 9, 'dominik': 10, 'nora': 11}


In [3]:
infer_file     = "/workdir2/home/sam/onto_oxford/relation_infer"
relations_file = "/workdir2/home/sam/onto_oxford/relations"

with open(relations_file,'r') as f:
    relation_number = 0
    for line in f:
        relation_number += 1
        
whole_dict     = {new_list: [] for new_list in range(relation_number)} 
wirelation     =  1
worelation     =  0
subject_index  =  1
relation_index =  2
object_index   =  3

with open(infer_file,'r') as f:
    for line in f:
        relation = (line.strip())
        if relation.split()[0]== "+":
            whole_dict[int(relation.split()[relation_index])].append([wirelation, int(relation.split()[subject_index]), int(relation.split()[object_index])])
        else:
            whole_dict[int(relation.split()[relation_index])].append([worelation, int(relation.split()[subject_index]), int(relation.split()[object_index])])

In [4]:
print (whole_dict[21][4])
# label, subject, object

[1, 0, 2]


In [5]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round((y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [6]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [16]:
class Siamese(nn.Module):
    
    def __init__(self, embedding, voca_size, hidden_size):
        
        self.voca_size   = voca_size
        self.embedding   = embedding
        self.hidden_size = hidden_size
        
        super(Siamese, self).__init__()
        self.conv = nn.Sequential(
            nn.Linear(self.voca_size, self.embedding),
            nn.Tanh(),
        )
        
        self.liner = nn.Sequential(nn.Linear(self.embedding*2, self.hidden_size), nn.Sigmoid())
        self.out   = nn.Sequential(nn.Linear(self.hidden_size, 1), nn.Sigmoid())
        

    def forward_one(self, x):
        x = self.conv(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        
        dis = torch.cat((out1, out2), dim=0)
        diss = self.liner(dis)
        out = self.out(diss)
        return out

In [17]:
loss_fn = torch.nn.BCEWithLogitsLoss(size_average=True)
embedding_dims = 50
hidden_size    = 10
learning_rate  = 0.01
net = Siamese(embedding_dims, vocabulary_size, hidden_size)
net.train()
optimizer = torch.optim.Adam(net.parameters(),lr = learning_rate )
optimizer.zero_grad()
train_loss = []
loss_val = 0
time_start = time.time()

In [19]:
correct     = 0
batch_print = 10
for iewr in range(1, 500):
    
    for label, su, ob in whole_dict[5]:
        optimizer.zero_grad()
        su_onehot = Variable(get_input_layer(su)).float()
        ob_onehot = Variable(get_input_layer(ob)).float()
        output = net.forward(su_onehot, ob_onehot)
        loss = loss_fn(output, torch.tensor([float(label)]))
        loss_val += loss.item()
        loss.backward()
        optimizer.step()
        
        output = (output>0.5).float()
        correct += (output == label).float().sum()
    if iewr % batch_print ==0 and iewr != 0:
        print(loss_val/len(whole_dict[23])/batch_print)
        loss_val = 0
        accuracy = 100 * (correct / batch_print) / len(whole_dict[21])
        print("Accuracy = {}".format(accuracy))
        correct = 0


1.1793130160205894
Accuracy = 95.83333587646484
0.6931472395857174
Accuracy = 95.83333587646484
0.6931471923987071
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.83333587646484
0.6931471824645996
Accuracy = 95.8333358

In [ ]:
net(Variable(get_input_layer(5)).float(), Variable(get_input_layer(10)).float())

In [ ]:
# 0 2
# 0 3
# 3 11
# 3 5
# 5 10
# 5 7
# 8 4
# 0 2
# 0 3
# 3 11
# 3 5
# 5 10

# print (net.conv[0].weight.cpu().detach().numpy())
# print ()
# print (net.conv[0].weight.cpu().detach().numpy()[:, 2])
# print ()
# print (net.conv[0].weight[:, 2])
# print ()
# print (Variable(get_input_layer(2)).float())